# 0 - Preparations

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats
import statistics

# Set the plotly theme
px.defaults.template = 'simple_white'

# Set the pandas display options
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Load the dataset
data = pd.read_csv('data/dataset_info_PPS.csv')
data.head(3)

,scan_id,bone,side,patient_id,exam,vgs,x_shape,y_shape,z_shape,max_val,...,mean_val,med_val,std_val,n_class_0,n_class_1,n_class_2,prop_class_0,prop_class_1,prop_class_2,image_path
0,D0004037,Radius,R,A01-0253,Ex01,VGS3,322,459,168,5824,...,348,99,866,14749962,7346617,2733485,0.59,0.30,0.11,/data/trainers/PPS_NPZs/Radius/A01-0253/Ex01/D...
1,D0004036,Radius,L,A01-0253,Ex01,VGS1,355,452,168,4902,...,356,97,860,17604824,6584447,2768008,0.65,0.24,0.10,/data/trainers/PPS_NPZs/Radius/A01-0253/Ex01/D...
2,D0004385,Radius,R,A01-0253,Ex02,VGS1,312,458,168,5437,...,379,104,900,13727102,7504448,2774978,0.57,0.31,0.12,/data/trainers/PPS_NPZs/Radius/A01-0253/Ex02/D...


In [3]:
data.shape

(1482, 21)

# 1 - Univariate Frequency Distribution
https://mciwing.github.io/statistics/univariate/Frequency/

In [4]:
data.columns

Index(['scan_id', 'bone', 'side', 'patient_id', 'exam', 'vgs', 'x_shape',
       'y_shape', 'z_shape', 'max_val', 'min_val', 'mean_val', 'med_val',
       'std_val', 'n_class_0', 'n_class_1', 'n_class_2', 'prop_class_0',
       'prop_class_1', 'prop_class_2', 'image_path'],
      dtype='object')

Values from dataset

| name | scale | note |
| - | - | - |
| scan_id | unique values | unique id for each image |
| bone | nominal | kind of bone, radius or tibia |
| side | nominal | side of the extremity, left or right |
| patient_id | unique values | unique id for each patient |
| exam | ordinal | id of the investigation, there is a chronological order |
| vgs | ordinal | type of quality of the scans (visual grading scale) |
| x_shape, y_shape, z_shape | numeric | dimensions in the corresponding dimension, number of voxels |
| max_val | numeric | maximal value (intensity) of voxels |
| min_val | numeric | minimal value (intensity) of voxels |
| mean_val | numeric | mean value (intensity) of voxels |
| median_val | numeric | median value (intensity) of voxels |
| std_val | numeric | mean value (intensity) of voxels |
| n_class_0, n_class_1, n_class_2 | numeric | number of voxels for each class |
| prop_class_0, prop_class_1, prop_class_2 | numeric | proportion of voxels for the individual classes |
| image_path | unique values | filepath to the image files |

Calculated values

| name | scale | note |
| - | - | - |
| n_voxels | numeric | total number of voxels |

In [5]:
# calculate the total number of voxels
data['n_voxels'] = data['x_shape'] * data['y_shape'] * data['z_shape']

data.head(3)

,scan_id,bone,side,patient_id,exam,vgs,x_shape,y_shape,z_shape,max_val,...,med_val,std_val,n_class_0,n_class_1,n_class_2,prop_class_0,prop_class_1,prop_class_2,image_path,n_voxels
0,D0004037,Radius,R,A01-0253,Ex01,VGS3,322,459,168,5824,...,99,866,14749962,7346617,2733485,0.59,0.30,0.11,/data/trainers/PPS_NPZs/Radius/A01-0253/Ex01/D...,24830064
1,D0004036,Radius,L,A01-0253,Ex01,VGS1,355,452,168,4902,...,97,860,17604824,6584447,2768008,0.65,0.24,0.10,/data/trainers/PPS_NPZs/Radius/A01-0253/Ex01/D...,26957280
2,D0004385,Radius,R,A01-0253,Ex02,VGS1,312,458,168,5437,...,104,900,13727102,7504448,2774978,0.57,0.31,0.12,/data/trainers/PPS_NPZs/Radius/A01-0253/Ex02/D...,24006528


### Nominal

In [6]:
# create a barplot with the frequency of locations
location_freq = data['bone'].value_counts().reset_index()
location_freq.columns = ['bone', 'count']  # rename columns for clarity

fig = px.bar(
    location_freq,
    x='bone',
    y='count',
    title='Frequency of bones in the dataset',
    width=500,
    )
fig.update_xaxes(title_text='bone')
fig.update_yaxes(title_text='Count')

fig.show()

In [7]:
# create a pie chart with the frequency of sides
side_freq = data['side'].value_counts().reset_index()
side_freq.columns = ['side', 'count']  # rename columns for clarity

fig = px.pie(
    side_freq,
    names='side',
    values='count',
    title='Frequency of sides',
    width=500,
    )

fig.show()

### Ordinal

In [8]:
# create a barplot with the frequency of quality
quality_freq = data['vgs'].value_counts().reset_index()
quality_freq.columns = ['vgs', 'count']  # rename columns for clarity

fig = px.bar(
    quality_freq,
    x='vgs',
    y='count',
    title='Frequency of image quality',
    width=500,
    )

fig.show()

In [9]:
# create a barplot with the frequency of quality
quality_freq = data['exam'].value_counts().reset_index()
quality_freq.columns = ['exam', 'count']  # rename columns for clarity
quality_freq = quality_freq.sort_values(by='exam')

fig = px.bar(
    quality_freq,
    x='exam',
    y='count',
    title='Frequency of exams',
    width=500,
    )

fig.show()

### Numeric

In [10]:
fig = px.histogram(
    data,
    x='n_voxels',
    nbins=50,
    title='Distribution of the number of voxels',
    width=500,
)

fig.show()

In [11]:
fig = px.histogram(
    data,
    x='n_voxels',
    nbins=50,
    facet_col='bone',
    title='Distribution of the number of voxels per bone',
    width=800,
)

fig.show()

> The distributions were calculated and displayed for the various data types.

# 2 - Measure of Central Tendency
https://mciwing.github.io/statistics/univariate/CentralTend/

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482 entries, 0 to 1481
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   scan_id       1482 non-null   object 
 1   bone          1482 non-null   object 
 2   side          1482 non-null   object 
 3   patient_id    1482 non-null   object 
 4   exam          1482 non-null   object 
 5   vgs           1482 non-null   object 
 6   x_shape       1482 non-null   int64  
 7   y_shape       1482 non-null   int64  
 8   z_shape       1482 non-null   int64  
 9   max_val       1482 non-null   int64  
 10  min_val       1482 non-null   int64  
 11  mean_val      1482 non-null   int64  
 12  med_val       1482 non-null   int64  
 13  std_val       1482 non-null   int64  
 14  n_class_0     1482 non-null   int64  
 15  n_class_1     1482 non-null   int64  
 16  n_class_2     1482 non-null   int64  
 17  prop_class_0  1482 non-null   float64
 18  prop_class_1  1482 non-null 

### Nominal

In [13]:
print('side')
print('----------------')

print(f'Mode: {data['side'].mode().values[0]}')

side
----------------
Mode: L


### Ordinal

In [14]:
print('vgs')
print('----------------')

print(f'Mode:   {data['vgs'].mode().values[0]}')

# keep only the last character of the vgs column and convert it to an integer to calculate the median
data['vgs_int'] = data['vgs'].str[-1].astype(int)
vgs_median = int(data['vgs_int'].median())
vgs_median = f'VGS{vgs_median}'

print(f'Median: {vgs_median}')


vgs
----------------
Mode:   VGS1
Median: VGS1


### Numeric

In [15]:
print('n_voxels')
print('----------------')

print(f'Mode:   {round(data['n_voxels'].mode().values[0], 2)}')
print(f'Median: {round(data['n_voxels'].median(), 2)}')
print(f'Mean:   {round(data['n_voxels'].mean(), 2)}')

n_voxels
----------------
Mode:   57879360
Median: 48579888.0
Mean:   51476226.4


In [16]:
# create a function to calculate the mode, median and mean for a given column
def calculate_measure_central_tendency(data, column):
    print(column)
    print('----------------')

    print(f'Mode:   {round(data[column].mode().values[0], 2)}')
    print(f'Median: {round(statistics.median(data[column]), 2)}')
    print(f'Mean:   {round(data[column].mean(), 2)}')

calculate_measure_central_tendency(data, 'n_voxels')

n_voxels
----------------
Mode:   57879360
Median: 48579888.0
Mean:   51476226.4


In [17]:
calculate_measure_central_tendency(data, 'x_shape')
print()
calculate_measure_central_tendency(data, 'y_shape')

x_shape
----------------
Mode:   406
Median: 517.5
Mean:   528.98

y_shape
----------------
Mode:   604
Median: 570.0
Mean:   568.28


In [18]:
data[['x_shape', 'y_shape', 'z_shape']].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
x_shape,1482.00,528.98,147.32,226.00,399.00,517.50,658.00,983.00
y_shape,1482.00,568.28,64.00,365.00,530.00,570.00,610.00,872.00
z_shape,1482.00,168.00,0.03,167.00,168.00,168.00,168.00,168.00


> Depending on the type of data, the measures of central tendency were calculated for several examples.

# 3 - Measure of Dispersion
https://mciwing.github.io/statistics/univariate/Dispersion/

In [19]:
data_numeric = data.select_dtypes(include=['number'])
data_numeric.head(3)

,x_shape,y_shape,z_shape,max_val,min_val,mean_val,med_val,std_val,n_class_0,n_class_1,n_class_2,prop_class_0,prop_class_1,prop_class_2,n_voxels,vgs_int
0,322,459,168,5824,-2215,348,99,866,14749962,7346617,2733485,0.59,0.30,0.11,24830064,3
1,355,452,168,4902,-1643,356,97,860,17604824,6584447,2768008,0.65,0.24,0.10,26957280,1
2,312,458,168,5437,-2294,379,104,900,13727102,7504448,2774978,0.57,0.31,0.12,24006528,1


### Range

In [20]:
# define a function to calculate the range for a given column
def calculate_range(data, column):
    return float(data[column].max() - data[column].min())

calculate_range(data, 'x_shape')

757.0

In [21]:
# Create a DataFrame to store measures of dispersion
measure_dispersion_df = pd.DataFrame(index=data_numeric.columns)

# Calculate range for all numeric columns
measure_dispersion_df['range'] = data_numeric.apply(lambda x: calculate_range(data_numeric, x.name))

measure_dispersion_df

,range
x_shape,757.00
y_shape,507.00
z_shape,1.00
max_val,11396.00
min_val,16198.00
mean_val,931.00
med_val,1035.00
std_val,599.00
n_class_0,52703001.00
n_class_1,78068752.00


### Interquartile Range

In [22]:
# define a function to calculate the interquartile range for a given column
def calculate_iqr(data, column):
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    return float(q3 - q1)

calculate_iqr(data, 'x_shape')

259.0

In [23]:
# calculate IQR for all numeric columns
measure_dispersion_df['iqr'] = data_numeric.apply(lambda x: calculate_iqr(data_numeric, x.name))
measure_dispersion_df

,range,iqr
x_shape,757.00,259.00
y_shape,507.00,80.00
z_shape,1.00,0.00
max_val,11396.00,671.25
min_val,16198.00,520.75
mean_val,931.00,140.00
med_val,1035.00,85.50
std_val,599.00,138.75
n_class_0,52703001.00,8362563.00
n_class_1,78068752.00,18889085.75


### Variance

In [24]:
# define a function to calculate the variance for a given column
def calculate_variance(data, column, round_to=3):
    return float(round(data[column].var(), round_to))

calculate_variance(data, 'x_shape')

21702.191

In [25]:
# calculate variance for all numeric columns
measure_dispersion_df['variance'] = data_numeric.apply(lambda x: calculate_variance(data_numeric, x.name))
measure_dispersion_df

,range,iqr,variance
x_shape,757.00,259.00,21702.19
y_shape,507.00,80.00,4096.03
z_shape,1.00,0.00,0.00
max_val,11396.00,671.25,597297.53
min_val,16198.00,520.75,431871.65
mean_val,931.00,140.00,10772.51
med_val,1035.00,85.50,4520.26
std_val,599.00,138.75,10479.49
n_class_0,52703001.00,8362563.00,41700070941099.95
n_class_1,78068752.00,18889085.75,128801804599273.75


### Standard Deviation

In [26]:
# define a function to calculate the standard deviation for a given column
def calculate_standard_deviation(data, column, round_to=3):
    return float(round(data[column].std(), round_to))

calculate_standard_deviation(data, 'x_shape')

147.317

In [27]:
# calculate standard deviation for all numeric columns
measure_dispersion_df['std_deviation'] = data_numeric.apply(lambda x: calculate_standard_deviation(data_numeric, x.name))
measure_dispersion_df

,range,iqr,variance,std_deviation
x_shape,757.00,259.00,21702.19,147.32
y_shape,507.00,80.00,4096.03,64.00
z_shape,1.00,0.00,0.00,0.03
max_val,11396.00,671.25,597297.53,772.85
min_val,16198.00,520.75,431871.65,657.17
mean_val,931.00,140.00,10772.51,103.79
med_val,1035.00,85.50,4520.26,67.23
std_val,599.00,138.75,10479.49,102.37
n_class_0,52703001.00,8362563.00,41700070941099.95,6457559.21
n_class_1,78068752.00,18889085.75,128801804599273.75,11349088.27


### Coefficient of Variation

In [28]:
data.columns

Index(['scan_id', 'bone', 'side', 'patient_id', 'exam', 'vgs', 'x_shape',
       'y_shape', 'z_shape', 'max_val', 'min_val', 'mean_val', 'med_val',
       'std_val', 'n_class_0', 'n_class_1', 'n_class_2', 'prop_class_0',
       'prop_class_1', 'prop_class_2', 'image_path', 'n_voxels', 'vgs_int'],
      dtype='object')

In [29]:
# define a function to calculate the coefficient of variation for a given column
def calculate_coefficient_of_variation(data, column, round_to=3):
    std_dev = data[column].std()
    mean = data[column].mean()
    return float(round(std_dev / mean, round_to))

calculate_coefficient_of_variation(data, 'min_val')

-0.252

In [30]:
# calculate coefficient of variation for all numeric columns
measure_dispersion_df['coeff_variation'] = data_numeric.apply(lambda x: calculate_coefficient_of_variation(data_numeric, x.name))
measure_dispersion_df

,range,iqr,variance,std_deviation,coeff_variation
x_shape,757.00,259.00,21702.19,147.32,0.28
y_shape,507.00,80.00,4096.03,64.00,0.11
z_shape,1.00,0.00,0.00,0.03,0.00
max_val,11396.00,671.25,597297.53,772.85,0.14
min_val,16198.00,520.75,431871.65,657.17,-0.25
mean_val,931.00,140.00,10772.51,103.79,0.24
med_val,1035.00,85.50,4520.26,67.23,0.36
std_val,599.00,138.75,10479.49,102.37,0.12
n_class_0,52703001.00,8362563.00,41700070941099.95,6457559.21,0.28
n_class_1,78068752.00,18889085.75,128801804599273.75,11349088.27,0.48


> The measures of disperson were determined for all numerical data and summarised in a dataframe. It is noticeable that the coefficient of variation for 'min_val' is negative, as all values and therefore also the mean value are negative. It can also be clearly seen that the coefficient of variation, unlike the others, is not dependent on the range of values and therefore enables a better comparison between the different columns.

# 4 - Bivariate Frequency Distribution
https://mciwing.github.io/statistics/bivariate/Frequency/

### Histogram

In [31]:
data.head(3)

,scan_id,bone,side,patient_id,exam,vgs,x_shape,y_shape,z_shape,max_val,...,std_val,n_class_0,n_class_1,n_class_2,prop_class_0,prop_class_1,prop_class_2,image_path,n_voxels,vgs_int
0,D0004037,Radius,R,A01-0253,Ex01,VGS3,322,459,168,5824,...,866,14749962,7346617,2733485,0.59,0.30,0.11,/data/trainers/PPS_NPZs/Radius/A01-0253/Ex01/D...,24830064,3
1,D0004036,Radius,L,A01-0253,Ex01,VGS1,355,452,168,4902,...,860,17604824,6584447,2768008,0.65,0.24,0.10,/data/trainers/PPS_NPZs/Radius/A01-0253/Ex01/D...,26957280,1
2,D0004385,Radius,R,A01-0253,Ex02,VGS1,312,458,168,5437,...,900,13727102,7504448,2774978,0.57,0.31,0.12,/data/trainers/PPS_NPZs/Radius/A01-0253/Ex02/D...,24006528,1


In [32]:
fig = px.density_heatmap(
    data,
    x='n_voxels',
    y='bone',
    title='Density heatmap of the number of voxels per bone',
    color_continuous_scale='Inferno',
    histnorm='percent',
    width=500,
    )

fig.show()

> The heatmap shows that the distribution of the number of voxels (volume) differs significantly depending on the type of bone. The volume of the tibia is clearly larger than that of the radius.

In [33]:
fig = px.density_heatmap(
    data,
    x='x_shape',
    y='y_shape',
    nbinsx=50,
    nbinsy=50,
    title='Density heatmap of x and y sizes',
    color_continuous_scale='Inferno',
    histnorm='percent',
    width=600,
    )

fig.show()

> The dependence of the x and y dimensions is clearly recognisable in the heatmap. The relationship will be examined further in the next chapters.

### Crosstab

In [34]:
pd.crosstab(
    data['bone'],
    data['vgs'],
    margins=True,
    margins_name='Total',
    )

vgs,VGS1,VGS2,VGS3,VGS4,VGS5,Total
bone,,,,,,
Radius,311,231,106,64,29,741
Tibia,514,125,63,24,15,741
Total,825,356,169,88,44,1482


In [35]:
pd.crosstab(
    data['bone'], 
    data['vgs'],
    margins=True,
    margins_name='Total',
    normalize='all',
    ).round(3)

vgs,VGS1,VGS2,VGS3,VGS4,VGS5,Total
bone,,,,,,
Radius,0.21,0.16,0.07,0.04,0.02,0.50
Tibia,0.35,0.08,0.04,0.02,0.01,0.50
Total,0.56,0.24,0.11,0.06,0.03,1.00


### Conditional frequency

In [36]:
cond_frequency_df = pd.crosstab(
    data['bone'], 
    data['vgs'],
    margins=True,
    margins_name='Total',
    normalize='index',
    ).round(3)

# add a column with the total values for each row
cond_frequency_df['Total'] = cond_frequency_df.sum(axis=1)

cond_frequency_df

vgs,VGS1,VGS2,VGS3,VGS4,VGS5,Total
bone,,,,,,
Radius,0.42,0.31,0.14,0.09,0.04,1.00
Tibia,0.69,0.17,0.09,0.03,0.02,1.00
Total,0.56,0.24,0.11,0.06,0.03,1.00


In [37]:
cond_frequency_df = pd.crosstab(
    data['bone'], 
    data['vgs'],
    margins=True,
    margins_name='Total',
    normalize='columns',
    ).round(3)

# add a row with the total values for each column
cond_frequency_df.loc['Total'] = cond_frequency_df.sum()

cond_frequency_df

vgs,VGS1,VGS2,VGS3,VGS4,VGS5,Total
bone,,,,,,
Radius,0.38,0.65,0.63,0.73,0.66,0.50
Tibia,0.62,0.35,0.37,0.27,0.34,0.50
Total,1.00,1.00,1.00,1.00,1.00,1.00


> The various crosstabs show that there are significantly more images with good quality (VGS1) for the radius compared to the tibia. Since the quality of the scan is strongly dependent on the movement during the scan, this could be due to the fact that the leg (tibia) can be fixed better than the arm (radius).

# 5 - Measure of Correlation
https://mciwing.github.io/statistics/bivariate/Correlation/

### Covariance

In [38]:
# define a function to calculate the covariance between two columns
def calculate_covariance(data, column1, column2, round_to=3):
    return float(round(data[column1].cov(data[column2]), round_to))

calculate_covariance(data, 'x_shape', 'y_shape')

5802.073

### Pearson Correlation Coefficient

In [39]:
# define a function to calculate the pearson correlation coefficient between two columns
def calculate_pearson_correlation(data, column1, column2, round_to=3):
    return float(round(data[column1].corr(data[column2], method='pearson'), round_to))

calculate_pearson_correlation(data, 'x_shape', 'y_shape')

0.615

### Spearman Correlation Coefficient

In [40]:
# define a function to calculate the spearman correlation coefficient between two columns
def calculate_spearman_correlation(data, column1, column2, round_to=3):
    return float(round(data[column1].corr(data[column2], method='spearman'), round_to))

calculate_spearman_correlation(data, 'x_shape', 'y_shape')

0.647

In [41]:
# define a function to calculate all three correlation coefficients between two columns
def calculate_correlation_coefficients(data, column1, column2):
    print(f'Covariance: {calculate_covariance(data, column1, column2)}')
    print(f'Pearson:    {calculate_pearson_correlation(data, column1, column2)}')
    print(f'Spearman:   {calculate_spearman_correlation(data, column1, column2)}')

calculate_correlation_coefficients(data, 'x_shape', 'y_shape')

Covariance: 5802.073
Pearson:    0.615
Spearman:   0.647


> The different measures of correlations are calculated and combined into one function to calculate them for a number of numerical variables at the same time.

### Scatter Plot

#### Minimum vs. maximum intensity

In [42]:
fig = px.scatter(
    data,
    x='min_val',
    y='max_val',
    facet_col='bone',
    title='Minimum vs. maximum intensity values per bone',
    labels={'min_val': 'Minimum intensity value', 'max_val': 'Maximum intensity value'},
    width=800,
)

fig.show()

In [43]:
calculate_correlation_coefficients(data, 'min_val', 'max_val')

Covariance: -406927.45
Pearson:    -0.801
Spearman:   -0.694


> The area of interest is difficult to recognise due to the outliers. These have been removed for better visualisation. These images should be viewed separately. This is not possible in the present work as the raw data cannot be shared. The outliers are therefore excluded for further calculation.

In [44]:
data[data['max_val'] > 10000]['scan_id']

393    D0005448
396    D0004734
669    D0004406
670    D0004405
696    D0005851
Name: scan_id, dtype: object

In [45]:
# Filter out the outliers for better visualization
fig = px.scatter(
    data[data['max_val'] < 10000],
    x='min_val',
    y='max_val',
    facet_col='bone',
    title='Minimum vs. maximum intensity values per bone',
    labels={'min_val': 'Minimum intensity value', 'max_val': 'Maximum intensity value'},
    width=800,
)

fig.show()

In [46]:
calculate_correlation_coefficients(data[data['max_val'] < 10000], 'min_val', 'max_val')

Covariance: -139606.139
Pearson:    -0.701
Spearman:   -0.691


> A negative correlation is recognisable, meaning that images with a higher maximum value also have a lower minimum value and therefore a higher overall span. There is no recognisable difference between radius and tibia.

In [47]:
calculate_correlation_coefficients(data, 'min_val', 'max_val')

Covariance: -406927.45
Pearson:    -0.801
Spearman:   -0.694


> It can be seen that the amount of the correlation is greater if the outliers are not excluded. this seems plausible, as these lie on the line in the scatterplot.

#### X-Dimension vs. Y-Dimension

In [48]:
px.scatter(
    data,
    x='x_shape',
    y='y_shape',
    color='bone',
    opacity=0.2,
    trendline='ols',
    title='X vs. Y dimensions',
    labels={'x_shape': 'x-size / voxel', 'y_shape': 'Y-size / voxel'},
    width=500,
)

> With radius, the trend line is steeper, which means that small x-dimensions are assigned to larger y-dimensions, which in turn corresponds to a more rectangular shape of the images. The images of the tibia have a more quardatic shape, which is evident from the trend line running at an angle of almost 45 degrees.

In [49]:
print('Total dataset')
print('----------------')
calculate_correlation_coefficients(data, 'x_shape', 'y_shape')

Total dataset
----------------
Covariance: 5802.073
Pearson:    0.615
Spearman:   0.647


In [50]:
print('Radius')
print('----------------')
calculate_correlation_coefficients(data[data['bone']=='Radius'], 'x_shape', 'y_shape')
print()
print('Tibia')
print('----------------')
calculate_correlation_coefficients(data[data['bone']=='Tibia'], 'x_shape', 'y_shape')

Radius
----------------
Covariance: 2511.952
Pearson:    0.822
Spearman:   0.821

Tibia
----------------
Covariance: 3749.638
Pearson:    0.856
Spearman:   0.854


> It can be seen that the correlation is significantly greater when the bones are considered separately. as is also shown in the scatterplot.

# 6 - Probability
https://mciwing.github.io/statistics/probability/General/

In [51]:
data['x_shape'].describe()

count   1482.00
mean     528.98
std      147.32
min      226.00
25%      399.00
50%      517.50
75%      658.00
max      983.00
Name: x_shape, dtype: float64

In [52]:
# show the distribution of the number of voxels per bone
fig = px.histogram(
    data,
    x='x_shape',
    facet_row ='bone',
    title='Distribution of the x_shape per bone',
    opacity=0.7,
    histnorm='probability density',
    width=600,
    height=600,
    )

fig.show()

> The distributions of the data differ between radius and tibia. further analysis is limited to the tibia; all steps can also be applied to radius and all other variables.

In [53]:
# fit a normal distribution to the data
mu, std = stats.norm.fit(data[data['bone']=='Tibia']['x_shape'])
mu, std

(np.float64(662.8070175438596), np.float64(69.6836635101856))

In [54]:
# Generate data from the fitted distribution to plot the distribution
x = np.arange(
    0,
    1001,
    10,
    )

pdf = stats.norm.pdf(x, mu, std)
cdf = stats.norm.cdf(x, mu, std)
df = pd.DataFrame({'x': x, 'pdf': pdf, 'cdf': cdf})

# Plot the fitted distribution
fig = px.line(
    df,
    x='x',
    y='pdf',
    title='Fitted normal distribution for the x_shape of the Tibia bone',
    width=800,
    )

# add the histogram to the plot
fig.add_histogram(
    x=data[data['bone']=='Tibia']['x_shape'],
    histnorm='probability density',
    xbins=dict(size=10),
    opacity=0.4,
    )

# dont show the histogram legend
fig.update_traces(showlegend=False)

# dont show the x-axis title
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='PDF')

fig.show()

> For further analysis, the normal distribution describes the data sufficiently. By using appropriate libraries (https://erdogant.github.io/distfit/pages/html/index.html), several distributions can be analysed and those with the best match can be selected.

In [55]:
upper_limit = 800
lower_limit = 500

# calculate the probability of having a x_shape value of 'lower_limit' or less
print(f'Propabyility of having a x_shape value of {lower_limit} or less: {round(stats.norm.cdf(lower_limit, mu, std), 2)}')

# calculate the probability of having a x_shape value of 'upper_limit' or more
print(f'Propabyility of having a x_shape value of {upper_limit} or more: {round(1 - stats.norm.cdf(upper_limit, mu, std), 2)}')

Propabyility of having a x_shape value of 500 or less: 0.01
Propabyility of having a x_shape value of 800 or more: 0.02


In [56]:
upper_limit = 700
lower_limit = 600

# calculate the probability of having a x_shape value between 'lower_limit' and 'upper_limit'
prob_between = round(stats.norm.cdf(upper_limit, mu, std) - stats.norm.cdf(lower_limit, mu, std), 2)
print(f'Propabyility of having a x_shape value between {lower_limit} and {upper_limit}: {prob_between}')

Propabyility of having a x_shape value between 600 and 700: 0.52


> The propability can be determined whether and how much data is below or above a limit value. An application for the measurement is the cropping or expansion of the image data for the training of a model to determine how much data is affected.

# 7 - Sampling
https://mciwing.github.io/statistics/probability/Sampling/

In [57]:
data.head(3)

,scan_id,bone,side,patient_id,exam,vgs,x_shape,y_shape,z_shape,max_val,...,std_val,n_class_0,n_class_1,n_class_2,prop_class_0,prop_class_1,prop_class_2,image_path,n_voxels,vgs_int
0,D0004037,Radius,R,A01-0253,Ex01,VGS3,322,459,168,5824,...,866,14749962,7346617,2733485,0.59,0.30,0.11,/data/trainers/PPS_NPZs/Radius/A01-0253/Ex01/D...,24830064,3
1,D0004036,Radius,L,A01-0253,Ex01,VGS1,355,452,168,4902,...,860,17604824,6584447,2768008,0.65,0.24,0.10,/data/trainers/PPS_NPZs/Radius/A01-0253/Ex01/D...,26957280,1
2,D0004385,Radius,R,A01-0253,Ex02,VGS1,312,458,168,5437,...,900,13727102,7504448,2774978,0.57,0.31,0.12,/data/trainers/PPS_NPZs/Radius/A01-0253/Ex02/D...,24006528,1


In [58]:
# calculate the mean proportion of each class
mean_prop = data[['prop_class_0', 'prop_class_1', 'prop_class_2']].mean()
mean_prop

prop_class_0   0.46
prop_class_1   0.44
prop_class_2   0.09
dtype: float64

In [59]:
n_samples = data.shape[0]
n_samples

1482

In [60]:
# take a sample of the data and calculate the mean proportion of each class
# repeat this process for different sample sizes and store the results in a DataFrame
mean_prop_df = pd.DataFrame()

for n_sample in range(1, n_samples + 1):

    mean_sample = pd.DataFrame([data[['prop_class_0', 'prop_class_1', 'prop_class_2']].sample(n=n_sample).mean()])
    mean_sample['n'] = n_sample

    mean_prop_df = pd.concat([mean_prop_df, mean_sample], ignore_index=True)

mean_prop_df

,prop_class_0,prop_class_1,prop_class_2,n
0,0.47,0.39,0.14,1
1,0.45,0.44,0.11,2
2,0.48,0.42,0.10,3
3,0.44,0.44,0.11,4
4,0.43,0.46,0.11,5
...,...,...,...,...
1477,0.46,0.44,0.09,1478
1478,0.46,0.44,0.09,1479
1479,0.46,0.44,0.09,1480
1480,0.46,0.44,0.09,1481


In [61]:
# create a line plot with the mean proportion of each class depending on the number of samples taken
fig = px.line(
    mean_prop_df,
    x='n',
    y=['prop_class_0', 'prop_class_1', 'prop_class_2'],
    title='Mean proportions of each class',
    labels={'value': 'Mean proportion', 'n': 'Number of samples'},
    width=800,
)

fig.show()

In [62]:
# calculate the relative difference from the mean proportion of each class
diff_from_mean = (mean_prop_df - mean_prop).abs() / mean_prop * 100
diff_from_mean['n'] = mean_prop_df['n']

fig = px.line(
    diff_from_mean,
    x='n',
    y=['prop_class_0', 'prop_class_1', 'prop_class_2'],
    title='Relative difference from the mean proportion of each class',
    labels={'value': 'Difference / %', 'n': 'Number of samples'},
    width=800,
)

fig.show()

> To calculate the proportion of individual classes in the data set, this can be done by using samples. To determine the number of voxels of the individual images, these must be loaded. For the present application, this takes several minutes for the entire data set. Here it can be advantageous to use a sample. as can be seen in the two diagrams, acceptable results are already achieved with n=200 by sampling.